In [2]:
from main import main
from utils import LoadData, LoadModel, load_data
import argparse
import numpy as np
import time
import copy
import os
import tempfile

import pandas as pd
# pyomo for optimization
import pyomo.environ as pyo

# pytorch for training neural network
import torch.onnx
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

# omlt for interfacing our neural network with pyomo
import onnx
from omlt import OffsetScaling, OmltBlock
from omlt.io.onnx import (
    load_onnx_neural_network_with_bounds,
    write_onnx_model_with_bounds,
    load_onnx_neural_network,
)
from omlt.neuralnet import (FullSpaceNNFormulation, 
    ReluComplementarityFormulation, 
    ReluPartitionFormulation,
    ReducedSpaceNNFormulation)

2025-01-12 11:11:43.669761: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 11:11:43.847760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736701903.914959    5138 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736701903.934900    5138 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 11:11:44.096298: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def add_arguments(model,model_id,system,scenario):    
    if system == 'membrane':
        args = argparse.Namespace(
            input_dim=7,
            hidden_dim=32,
            hidden_num=2,
            z0_dim=8,
            optimizer='adam',
            epochs=1000,
            batch_size=16,
            lr=1e-4,
            mu=1,
            max_subiter=500,
            eta=0.8,
            sigma=2,
            mu_safe=1e+9,
            dtype=32,
            dataset_path='/home/andresfel9403/KKThNN/KKThPINN/benchmark_membrane.csv',
            val_ratio=0.2,
            job='train',
            runs=10)
    else:
        pass

    args.model = model
    args.model_id = model_id
    args.dataset_type = system
    args.scenario = scenario
        
    
    if args.model == 'NN':
        args.loss_type = 'MSE'
    elif args.model == 'PINN':
        args.loss_type = 'PINN'
    elif args.model == 'KKThPINN':
        args.loss_type = 'MSE'
    elif args.model == 'AugLagNN':
        args.loss_type = 'MSE'
    elif args.model == 'ECNN':
        args.loss_type = 'MSE'
    return args

In [5]:
inputs = ['F_MeOH',
    'F_DME',
    'F_H2O',
    'F_N2',
    'T_in',
    'P_in',
    'T_permeate']

outputs = [
    'F_MeOH_O',
    'F_DME_O',
    'F_H2O_O',
    'F_H2O_M_O',
    'F_N2_O',
    'T_Out',
    'T_Out_M',
    'P_Out'
]

columns = inputs+outputs

df = pd.read_csv("benchmark_membrane.csv", usecols = columns)


## Neural Network Training  

In [6]:
args_NN = add_arguments('NN','MembraneT_NN','membrane','demonstration')

In [7]:
main(args_NN)

type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204
Start Training...
epoch: 00050 loss_train: 0.01253 loss_val: 0.01299 violation_train: 10.24131 violation_val: 10.80998
epoch: 00100 loss_train: 0.00271 loss_val: 0.00300 violation_train: 2.26623 violation_val: 2.07504
epoch: 00150 loss_train: 0.00105 loss_val: 0.00152 violation_train: 0.25400 violation_val: 0.28112
epoch: 00200 loss_train: 0.00083 loss_val: 0.00130 violation_train: 0.25836 violation_val: 0.32832
epoch: 00250 loss_train: 0.00072 loss_val: 0.00115 violation_train: 0.18442 violation_val: 0.21387
epoch: 00300 loss_train: 0.00066 loss_val: 0.00106 violation_train: 0.18189 violation_val: 0.18695
epoch: 00350 loss_train: 0.00065 loss_val: 0.00101 violation_train: 0.18037 violation_val: 0.20561
epoch: 00400 loss_train: 0.00064 loss_val: 0.00100 violation_train: 0.22044 violation_val: 0.32950
epoch: 00450 loss_train: 0.00063 loss_val: 0.00100 v

/home/andresfel9403/KKThNN/KKThPINN/train.py:264: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


After defining the arguments depending on the number of inputs, hidden and output layers, we can train the model selecting the Neural Network model (NN or KKThPINN)

In [8]:
dataset, scaling = load_data(args_NN.dataset_path)
inputs = dataset[:,:7]
outputs = dataset[:,7:]

data = LoadData(args_NN)
model_NN = LoadModel(args_NN, data)
PATH = '/home/andresfel9403/KKThNN/models/membrane/NN/0.2/None_0.2_9.pth'
checkpoint = torch.load(PATH)
model_NN.load_state_dict(checkpoint['state_dict'])
#model_NN.eval()



type of A: torch.float32, type of B: torch.float32, type of b: torch.float32
train set size: 614, val set size: 204, test set size: 204


/tmp/ipykernel_5138/2941437368.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(PATH)


<All keys matched successfully>

In [9]:
x_factor = scaling.scale_[:7]
y_factor = scaling.scale_[7:]

print(x_factor)

[1.46831994e+01 1.04111445e+01 1.42231966e+01 9.99584502e+01
 7.26638589e+02 2.69926289e+06 1.76773269e+02]


In [10]:
scaler = scaler = OffsetScaling(
    offset_inputs={i: 0 for i in range(len(x_factor))},
    factor_inputs={i: x_factor[i] for i in range(len(x_factor))},
    offset_outputs={i: 0 for i in range(len(y_factor))},
    factor_outputs={i: y_factor[i] for i in range(len(y_factor))},
)

In [11]:
x_dummy = torch.from_numpy(inputs[0]).float()
ub = np.max(inputs, 0)
lb = np.min(inputs, 0)

scaled_input_bounds = {i: (lb[i], ub[i]) for i in range(len(inputs[0]))}
print(ub.dtype)

float64


In [12]:
def transfer_weights(original_model, modified_model):
    with torch.no_grad():
        for original_layer, modified_layer in zip(original_model.layers, modified_model.layers):
            if isinstance(original_layer, nn.Linear) and isinstance(modified_layer, nn.Linear):
                modified_layer.weight.copy_(original_layer.weight)
                modified_layer.bias.copy_(original_layer.bias)

In [13]:
def _create_onnx_model(data,args,model,file_path,x,input_bounds):
    args.model='NN'
    print('Saving standard model')
    modified_model = LoadModel(args,data)
    transfer_weights(model,modified_model)
    create_onnx_model(data,modified_model,file_path,x,input_bounds)

In [14]:
def create_onnx_model(data,model,file_path,x,input_bounds):
    
    torch.onnx.export(
        model,
        x,
        file_path,
        input_names=["input"],
        dynamo = False
    )
    write_onnx_model_with_bounds(file_path, None, input_bounds)
    print(f"Wrote PyTorch Onnx model to {file_path}")

In [15]:
_create_onnx_model(data,args_NN,model_NN,'NN_Membrane.onnx',x_dummy,scaled_input_bounds)

Saving standard model
Wrote PyTorch Onnx model to NN_Membrane.onnx


In [28]:
#with tempfile.NamedTemporaryFile(suffix=".onnx", delete=False) as f:
    # export neural network to ONNX
torch.onnx.export(
    model_NN,
    x_dummy,
    'intento.onnx',
    input_names=["input"],
    dynamo = False
)
# write ONNX model and its bounds using OMLT
write_onnx_model_with_bounds('intento.onnx', None, scaled_input_bounds)
onnx_model = onnx.load('intento.onnx')
# load the network definition from the ONNX model
#network_definition = load_onnx_neural_network_with_bounds('intento.onnx')
network_definition = load_onnx_neural_network(onnx = onnx_model,  scaling_object=scaler, input_bounds= scaled_input_bounds) 

In [16]:
network_definition_NN = load_onnx_neural_network_with_bounds('NN_Membrane.onnx') 


In [17]:
formulation_NN = FullSpaceNNFormulation(network_definition_NN)

In [20]:
for layer_id, layer in enumerate(network_definition_NN.layers):
    print(f"{layer_id}\t{layer}\t{layer.activation}")

0	InputLayer(input_size=[7], output_size=[7])	linear
1	DenseLayer(input_size=[7], output_size=[32])	relu
2	DenseLayer(input_size=[32], output_size=[32])	relu
3	DenseLayer(input_size=[32], output_size=[8])	linear


In [21]:
model = pyo.ConcreteModel()

# create an OMLT block for the neural network and build its formulation
model.membrane= OmltBlock()
model.membrane.build_formulation(formulation_NN)

# model.sections = pyo.Set(initialize=["retentate", "permeate"], doc="sections in the model")
# model.components = pyo.Set(initialize=["MeOH", "DME", "H2O", "N2"], doc="Components in the model")


WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[0]' to a numeric value
`0` outside the bounds (0.002303625455624813, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[1]' to a numeric value
`0` outside the bounds (0.00045880834803283796, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[2]' to a numeric value
`0` outside the bounds (0.0022512226291679363, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING (W1002): Setting Var 'membrane.scaled_inputs[3]' to a numeric value
`0` outside the bounds (0.10018982303101381, 1.0).
    See also https://pyomo.readthedocs.io/en/stable/errors.html#w1002
WARNING 

In [89]:
inputs = ['F_MeOH',
    'F_DME',
    'F_H2O',
    'F_N2',
    'T_in',
    'P_in',
    'T_permeate']

outputs = [
    'F_MeOH_O',
    'F_DME_O',
    'F_H2O_O',
    'F_H2O_M_O',
    'F_N2_O',
    'T_Out',
    'T_Out_M',
    'P_Out'
]

Tin_idx = inputs.index('T_in')
Tm_idx = inputs.index('T_permeate')
F_MeOH_idx = inputs.index('F_MeOH')
F_DME_idx = inputs.index('F_DME')
F_H2O_idx = inputs.index('F_H2O')
F_N2_idx = inputs.index('F_N2')
P_in_idx = inputs.index('P_in')

def Tin_bounds(m):
    return (460/x_factor[Tin_idx],m.membrane.inputs[Tin_idx],1000/x_factor[Tin_idx])

def T_mem_bounds(m):
    return (283.15/x_factor[Tm_idx],m.membrane.inputs[Tm_idx],450/x_factor[Tm_idx])

def TFlow_bounds(m):
    FMeOh = m.membrane.inputs[F_MeOH_idx]*x_factor[F_MeOH_idx]
    FDME = m.membrane.inputs[F_DME_idx]*x_factor[F_DME_idx]
    FH2O = m.membrane.inputs[F_H2O_idx]*x_factor[F_H2O_idx]
    FN2 = m.membrane.inputs[F_N2_idx]*x_factor[F_N2_idx]
    Ftot = FMeOh + FDME + FH2O + FN2
    return (1, Ftot, 100)

def Flow_DME(m):

    return (0, m.membrane.inputs[F_DME_idx], 0.005)

def P_bounds(m):
    return (13e5/x_factor[P_in_idx], m.membrane.inputs[P_in_idx], 27e5)

model.Tin_constr = pyo.Constraint(rule = Tin_bounds)
model.Tm_constr = pyo.Constraint(rule = T_mem_bounds)
model.TFlow_constr = pyo.Constraint(rule = TFlow_bounds)
model.P_constr = pyo.Constraint(rule = P_bounds)
#model.Flow_DME = pyo.Constraint(rule = Flow_DME)





'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarCon

In [90]:
DME_O = model.membrane.outputs[outputs.index("F_DME_O")]*y_factor[outputs.index("F_DME_O")]


In [95]:
def objective_rule(m):
    prices = {
        "MeOH": 891*1e-6,  # Cost of Methanol in $/kg
        "DME": 2021.84*1e-6,   # Cost of Dimethyl Ether in $/kg
        "H2O": 0.29*1e-6,  # Cost of Water in $/kg
        "N2": 121.254*1e-6  # Cost of Nitrogen in $/kg
    }

    MW = {
        "MeOH": 32.04,  # MW of Methanol in g/mol
        "DME": 46.069,   # Cost of Dimethyl Ether in $/kg
        "H2O": 18.0152,  # Cost of Water in $/kg
        "N2": 28.0134  # Cost of Nitrogen in $/kg
    }

    DME_O = m.membrane.outputs[outputs.index("F_DME_O")]*y_factor[outputs.index("F_DME_O")]
    H2O_O = m.membrane.outputs[outputs.index("F_H2O_O")]*y_factor[outputs.index("F_H2O_O")]
    H2O_M_O = m.membrane.outputs[outputs.index("F_H2O_M_O")]*y_factor[outputs.index("F_H2O_M_O")]
    MeOH_In = m.membrane.inputs[inputs.index("F_MeOH")]*x_factor[inputs.index("F_MeOH")]
    N2_In = m.membrane.inputs[inputs.index("F_N2")]*x_factor[inputs.index("F_N2")]
    DME_In = m.membrane.inputs[inputs.index("F_DME")]*x_factor[inputs.index("F_DME")]

    product_profit = prices["DME"]*DME_O*MW["DME"] + prices["H2O"]*H2O_O*MW["H20"] + prices["H2O"]*H2O_M_O*MW["H20"]

    feedstock_cost = prices["MeOH"]*MeOH_In*MW["MeOH"] + prices["N2"]*N2_In*MW["N2"] + prices["DME"]*DME_In*MW["DME"]
    return product_profit - feedstock_cost

model.objective = pyo.Objective(rule = objective_rule, sense = pyo.maximize)


'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for Objective objective with
index None: KeyError: 'H20'
ERROR: Constructing component 'objective' from data=None failed:
        KeyError: 'H20'


KeyError: 'H20'

In [92]:
solver = pyo.SolverFactory("ipopt")
status = solver.solve(model, tee=True)

Ipopt 3.14.6: 
==> Warning: Treating 64 binary and 0 integer variables as continuous.


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     1524
Number of nonzeros in inequality constraint Jacobian.:      504
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:      241
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      225
                     variables with only upper bounds:        0
Total 

In [94]:
print("FA:", pyo.value(model.membrane.inputs[inputs.index("F_MeOH")])*x_factor[inputs.index("F_MeOH")])
print("FDME:", pyo.value(model.membrane.inputs[inputs.index("F_DME")])*x_factor[inputs.index("F_DME")])
print("FDME:", pyo.value(model.membrane.outputs[outputs.index("F_DME_O")])*y_factor[outputs.index("F_DME_O")])
print("FH2O:", pyo.value(model.membrane.outputs[outputs.index("F_H2O_O")])*y_factor[outputs.index("F_H2O_O")])
print("FH2OM:", pyo.value(model.membrane.outputs[outputs.index("F_H2O_M_O")])*y_factor[outputs.index("F_H2O_M_O")])
print("FN2:", pyo.value(model.membrane.inputs[inputs.index("F_N2")])*x_factor[inputs.index("F_N2")])
print("Tout:", pyo.value(model.membrane.outputs[outputs.index("T_Out")])*y_factor[outputs.index("T_Out")])
print(pyo.value(model.TFlow_constr))

FA: 0.2991536256509395
FDME: 0.043793769214874455
FDME: 3.506323355784384
FH2O: 1.7887266525660925
FH2OM: 5.088371816962074
FN2: 12.208628782651937
Tout: 738.6718774395302
14.40570949450101


In [ ]:
print(x_factor)